## Importing Libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import tree
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Import Data from Excel Sheet

In [2]:
file_name =  "Dataset.xls"
sheet =  'Heart Disease 1'

df = pd.read_excel(io=file_name, sheet_name=sheet)
df=df.head(100) #Select first 100 rows

# Data Processing

## Select the given attributes

In [3]:
#BMI, Smoking, PhysicalHealth, Race, Age
df = df[['BMI', 'Smoking', 'PhysicalHealth', 'Race', 'AgeCategory']]

df

,BMI,Smoking,PhysicalHealth,Race,AgeCategory
0,16.60,Yes,3,White,55-59
1,20.34,No,0,White,80 or older
2,26.58,Yes,20,White,65-69
3,24.21,No,0,White,75-79
4,23.71,No,28,White,40-44
...,...,...,...,...,...
95,19.49,No,0,White,45-49
96,24.02,No,0,White,70-74
97,28.35,Yes,20,White,65-69
98,24.37,No,0,White,55-59


## Check if there is any column with "nan" values

In [4]:
df.isnull().any()

BMI               False
Smoking           False
PhysicalHealth    False
Race              False
AgeCategory       False
dtype: bool

## Change the intervals to midpoint values

In [ ]:
for i in df['AgeCategory']:
    if i == '<25':
        df['AgeCategory'] = df['AgeCategory'].replace(['<25'],19.5)
    elif i == '25-34':
        df['AgeCategory'] = df['AgeCategory'].replace(['25-34'],29.5)
    elif i== '35-44':
        df['AgeCategory'] = df['AgeCategory'].replace(['35-44'],39.5)
    elif i== '45-54':
        df['AgeCategory'] = df['AgeCategory'].replace(['45-54'],49.5)
    elif i== '55-64':
        df['AgeCategory'] = df['AgeCategory'].replace(['55-64'],59.5)
    elif i== '65-74':
        df['AgeCategory'] = df['AgeCategory'].replace(['65-74'],69.5)
    elif i== '>74':
        df['AgeCategory'] = df['AgeCategory'].replace(['>74'],79.5)